In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from scripts import tools
from scripts import dataset
from scripts.config import *

%matplotlib inline

Joint_Classes = np.array(["Right ankle", "Right knee", "Right hip", "Left hip", "Left knee", "Left ankle", "Right wrist",
                  "Right elbow", "Right shoulder", "Left shoulder", "Left elbow", "Left wrist", "Neck", "Head top"])

MET_Classes = np.array(["walking about", "writing", "reading.seated", "typing", "lifting.packing-lifting", "lifting.packing-packing", 
                        "filling.seated", "filling.stand", "cooking", "house cleaning", "machine work.light", "machine work.sawing", 
                        "reclining", "seated.quiet", "sleeping", "standing.relaxed"])

tools.etc.set_GPU("2,3")

In [2]:
batch_size=96
learning_rate = 0.01
num_joints = len(Joint_Classes)
num_met = len(MET_Classes)

display_step = 100

PROJECT_ROOT = %pwd

train_csv = PROJECT_ROOT + "/dataset/train.csv"
test_csv = PROJECT_ROOT + "/dataset/test.csv"

if not os.path.isdir(checkpoint_path):
    os.mkdir(checkpoint_path)

In [5]:
#train_data = dataset.iterator(csv_file="./dataset/train_MET_L.csv", batch_size=batch_size, Shuffle=True)
test_data = dataset.met(csv_file="./dataset/test_MET_L.csv", Shuffle=True, one_hot = True)

[Processing Images & Coordinates]: 100%|██████████| 96/96 [00:01<00:00, 74.58it/s]


In [4]:
from models import alexnet

model = alexnet.alexnet(batch_size, (227,227,3), (11,))
score = model.fc8

train_layers = ['fc8', 'fc7', 'fc6']
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,labels=model.y_gt))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [5]:
with tf.name_scope("train"):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=1,rho=0.95,epsilon=1e-09).minimize(loss)
    
for var in var_list:
    tf.summary.histogram(var.name, var)
    
tf.summary.scalar('cross_entropy', loss)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(model.y_gt, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(filewriter_path)

saver = tf.train.Saver()

INFO:tensorflow:Summary name fc6/weights:0 is illegal; using fc6/weights_0 instead.
INFO:tensorflow:Summary name fc6/biases:0 is illegal; using fc6/biases_0 instead.
INFO:tensorflow:Summary name fc7/weights:0 is illegal; using fc7/weights_0 instead.
INFO:tensorflow:Summary name fc7/biases:0 is illegal; using fc7/biases_0 instead.
INFO:tensorflow:Summary name fc8/weights:0 is illegal; using fc8/weights_0 instead.
INFO:tensorflow:Summary name fc8/biases:0 is illegal; using fc8/biases_0 instead.


In [6]:
train_batches_per_epoch = int(np.floor(train_data.img_set.shape.as_list()[0]/batch_size))
num_val=test_data.img_set.shape.as_list()[0]
val_batches_per_epoch = int(np.floor(num_val/ batch_size if num_val > batch_size else num_val))

keep_prob = tf.placeholder(tf.float32)

num_epochs = 
next_batch = train_data.iterator.get_next()

In [40]:
with tf.device('/gpu:0'):
    with tqdm(total = num_epochs) as pbar:
        with tf.Session() as sess:
            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            # Add the model graph to TensorBoard
            writer.add_graph(sess.graph)

            # Loop over number of epochs
            for epoch in range(num_epochs):

                # Initialize iterator with the training dataset
                sess.run(train_data.init_op)

                train_acc = 0.
                train_loss = 0.
                train_count = 0
                for step in range(train_batches_per_epoch):
                    # get next batch of data
                    img_batch, label_batch = sess.run(next_batch)

                    # And run the training op
                    acc,cost,op=sess.run([accuracy,loss,optimizer], feed_dict={model.x: img_batch,
                                                  model.y_gt: label_batch,
                                                  keep_prob: 0.7})
                    # Generate summary with the current batch of data and write to file
                    if step % display_step == 0:
                        s = sess.run(merged_summary, feed_dict={model.x: img_batch,
                                                                model.y_gt: label_batch,
                                                                keep_prob: 0.7})

                        writer.add_summary(s, epoch*train_batches_per_epoch + step)
                    train_acc += acc
                    train_loss += cost
                    train_count += 1
                
                #"""
                # Validate the model on the entire validation set

                sess.run(test_data.init_op)
                test_acc = 0.
                test_loss = 0.
                test_count = 0
                for _ in range(val_batches_per_epoch):
                    acc,cost = sess.run([accuracy,loss], feed_dict={model.x: testtest.img_set,
                                                                  model.y_gt: np.eye(np.max(testtest.labels)+1)[testtest.labels].reshape(96,11),
                                                                  keep_prob: 0.7})
                    test_acc += acc
                    test_loss += cost
                    test_count += 1
                    
                pbar.set_description("[Epochs : "+str(epoch)+" ]"+
                                     " Train - Accuracy : "+str(train_acc/train_count if train_count !=0 else 0)+
                                     " Train - Loss : "+str(train_loss/train_count if train_count !=0 else 0)+
                                     " Test - Accuracy : "+str(test_acc/test_count if test_count !=0 else 0)+
                                     " Test - Loss : "+str(test_loss/test_count if test_count !=0 else 0))
                pbar.update(1)
                #"""
                # save checkpoint of the model
                checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
                save_path = saver.save(sess, checkpoint_name)

[Epochs : 299 ] Train - Accuracy : 0.9942129585478041 Train - Loss : 0.022381499111159227 Test - Accuracy : 0.3905164925381541 Test - Loss : 7.472269892692566: 100%|██████████| 300/300 [1:24:30<00:00, 20.53s/it]  
